In [1]:
import queue
import time
import os
import sys

from DataHandler.TradeLOBHourlyDataHandler import HistoricTradeLOBHourlyDataHandler
from Strategy.LeadLagArbitrageStrategy import LeadLagArbitrageStrategy
from Portfolio.LogPlotPortfolio import LogPlotPortfolio
from Execution.execution import SimulatedExecutionHandler

event_queue = queue.Queue()
data_handler = HistoricTradeLOBHourlyDataHandler(event_queue, 
                                           symbol_list=['btc_usdt','btc_usdt'],
                                           exchange_list=['binance','bybit'], 
                                           file_dir = 'data_sample/20240101/', 
                                           # file_dir = 'data_sample/',
                                           is_csv=False)
#sys.exit()

# 组合
portfolio = LogPlotPortfolio(event_queue, data_handler)
# 回测模拟成交器；如果是实盘这里就是算法交易模块
executor = SimulatedExecutionHandler(event_queue, data_handler) 
# 策略实例。实际应用中应该有多个策略实例
strategy = LeadLagArbitrageStrategy(event_queue, data_handler, portfolio, executor) 

cnt = 0
while True:
    # Update the trade/LOB (specific backtest code, as opposed to live trading)
    if data_handler.continue_backtest == True:
        data_handler.update_TradeLOB()
    else:
        break
    
    cnt+=1
    if cnt>2000: break

    # Handle the events
    while True:
        try:
            event = event_queue.get(False)
        except queue.Empty:
            break
        else:
            if event is not None:

                if event.type == 'MARKET':
                    # print('get market event', event)
                    strategy.on_market_event(event)
                    portfolio.on_market_event(event)
                    executor.on_market_event(event)

                elif event.type == 'ORDER':
                    print('get order event', event)
                    executor.on_order_event(event)

                elif event.type == 'FILL':
                    print('get fill event', event)
                    executor.on_fill_event(event)
                    portfolio.on_fill_event(event)
                    sys.exit()
                    strategy.on_order_fill(event)
                    #sys.exit()


backtest on:  ['btc_usdt_binance', 'btc_usdt_bybit']
/*----- start initialize the DataHandler -----*/


In [ ]:
executor.live_orders_on_exchange

{'btc_usdt_binance': [],
 'btc_usdt_bybit': [{'timestamp': 1704042004655, 'symbol': 'btc_usdt_bybit', 'order_id': 0, 'order_type': 'IOC', 'direction': 'BUY', 'quantity': 0.23473649707735345, 'price': 42600.959478000004}]}

In [ ]:
[i for i in executor.live_orders_on_exchange['btc_usdt_bybit'] if i.order_id != event.order_id]

[]